# Cython C++ QASM Simulator

## Using the simulator as a QISKit engine

We now import the cython simulator and use it as a qiskit backend

In [14]:
# Add qiskit development branch to path
# Once qiskit 0.6 is released this can use the pip qiskit
QISKIT_PATH = '~/git/qiskit-terra/'
import sys, os
sys.path.append(os.path.expanduser(QISKIT_PATH))

In [15]:
import numpy as np
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

# Import Cython QASM Simulator Backend
from qiskit_addon_qv import AerQvSimulator

# Backends
aer = AerQvSimulator()
aer.set_max_threads_shot(-1)

In [16]:
def random_circuit(num_qubits, num_gates):
    qr = qiskit.QuantumRegister(num_qubits)
    cr = qiskit.ClassicalRegister(num_qubits)
    circ = qiskit.QuantumCircuit(qr, cr)
    for j in range(num_gates):
        q = np.random.randint(0, num_qubits)
        
        if num_qubits == 1:
            r = np.random.randint(0, 3)
        else:
            r = np.random.randint(0, 3)
        
        if r == 0:
            circ.h(qr[q])
        elif r == 1:
            circ.s(qr[q])
        elif r == 2:
            circ.t(qr[q])
        elif r == 3:       
            circ.cx(qr[q], qr[(q + 1) % num_qubits])
    circ.measure(qr, cr)
    return circ

In [17]:
circ1 = random_circuit(5, 50)
result1 = qiskit.execute(circ1, aer, shots=1000).result()

In [18]:
result1.get_counts(circ1)

{'0x0': 15,
 '0x1': 82,
 '0x10': 3,
 '0x11': 15,
 '0x13': 25,
 '0x14': 2,
 '0x15': 12,
 '0x16': 3,
 '0x17': 17,
 '0x18': 3,
 '0x19': 11,
 '0x1a': 4,
 '0x1b': 18,
 '0x1c': 4,
 '0x1d': 11,
 '0x1e': 2,
 '0x1f': 14,
 '0x2': 15,
 '0x3': 107,
 '0x4': 7,
 '0x5': 96,
 '0x6': 15,
 '0x7': 105,
 '0x8': 15,
 '0x9': 72,
 '0xa': 14,
 '0xb': 95,
 '0xc': 15,
 '0xd': 97,
 '0xe': 14,
 '0xf': 92}

### Noise simulation

Simulation of an amplitude damping channel on single qubit gates

In [19]:
noise_model = {
    "errors": [
        {
            "type": "kraus",
            "operations": ["h", "t"],
            "probabilities": [1],
            "matrices": [np.array([[1, 0], [0, 0.5]]),
                        np.array([[0, 0.86602540378], [0, 0]])]
        }
    ]
}

In [20]:
aer.load_noise_model(noise_model)

In [21]:
circ2 = random_circuit(5, 50)
result2 = qiskit.execute(circ2, aer, shots=1000).result()
result2.get_data(circ2)

{'counts': {'0x0': 9,
  '0x1': 15,
  '0x10': 5,
  '0x11': 4,
  '0x13': 16,
  '0x17': 23,
  '0x1b': 1,
  '0x1f': 676,
  '0x3': 46,
  '0x7': 84,
  '0xb': 2,
  '0xf': 119}}